In [1]:
# will always need this 4 lines to start 
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext("local", "Cache")

22/03/02 20:29:00 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/02 20:29:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/02 20:29:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from pyspark import StorageLevel

In [17]:
# how to remove special chars
# we will remove all ascii
def to_ascii(text):
    import re
    output = re.sub(r"[^a-zA-z0-9 ]", "", text)
    #print(output)
    return output
    
text = "prince, don't"
to_ascii(text)

'prince dont'

In [36]:
wordCountRdd = sc.textFile("hdfs://localhost:9000/book-war-and-peace.txt")\
                    .map(lambda line: to_ascii(line))\
                    .map(lambda line: line.strip().lower())\
                    .map (lambda line: line.split(" "))\
                    .flatMap(lambda elements: elements)\
                    .filter (lambda word: word != "")\
                    .map (lambda word: (word, 1))\
                    .reduceByKey(lambda acc, value: acc + value)

In [37]:
# wordCountRdd.cache() # , call persit internally with MEMORY_ONLY

#wordCountRdd.persist(StorageLevel.MEMORY_AND_DISK) # we have many options
wordCountRdd.persist(StorageLevel.DISK_ONLY) # we have many options
wordCountRdd.take(10)

[('chapter', 367),
 ('i', 4089),
 ('well', 741),
 ('prince', 1886),
 ('so', 1852),
 ('genoa', 3),
 ('and', 22082),
 ('lucca', 2),
 ('are', 1259),
 ('now', 1305)]

In [40]:
# sort data by value
sortedRddAscending = wordCountRdd.sortBy(lambda kv: kv[1])
sortedRddAscending.take(10)

[('infamies', 1),
 ('yousit', 1),
 ('elite', 1),
 ('scarletliveried', 1),
 ('10annette', 1),
 ('grandfathers', 1),
 ('canceled', 1),
 ('woundup', 1),
 ('tease', 1),
 ('novosiltsevs', 1)]

In [41]:
# sortBy values but decending order
print(wordCountRdd.getNumPartitions())
sortedRddDecending = wordCountRdd.sortBy(lambda kv: kv[1], ascending=False)
sortedRddDecending.take(10)

1


[('the', 34395),
 ('and', 22082),
 ('to', 16636),
 ('of', 14872),
 ('a', 10464),
 ('he', 9807),
 ('in', 8744),
 ('his', 7967),
 ('that', 7798),
 ('was', 7328)]

In [42]:
wordCountRdd.saveAsTextFile("hdfs://localhost:9000/war-and-peace-1.txt")

In [43]:
sortedRddDecending.saveAsTextFile("hdfs://localhost:9000/war-and-peace-decending.txt")

In [45]:
sortedRddAscending.saveAsTextFile("hdfs://localhost:9000/war-and-peace-ascending.txt")